<a href="https://colab.research.google.com/github/darthgera123/RAG-Agents/blob/main/RAG_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG PDF
This notebook takes a pdf as input and leverages a RAG agent to answer user queries in natural language.

In [1]:
!pip install langchain pypdf openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s et

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [5]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.0 MB/s eta 0:00:00


## Import Libraries

In [4]:
import numpy as np
import json
import openai
import os
from google.colab import userdata
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pypdf
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import fitz

openai.api_key = userdata.get('OPEN_AI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_AI_API_KEY')
client = openai.Client()

## Extract Text

## Load files

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Create Database
Define functions that read the input pdfs, extract the contents, split into chunks and then based on defined embedding load it into a vector database

### Extract Contents

In [31]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file using PyMuPDF.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text from the PDF.
    """
    pdf = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf.page_count):
        page = pdf[page_num]
        text += page.get_text()
    pdf.close()
    return text

### Tokenize contents
Splits text into chunks

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks = text_splitter.create_documents([pdf_text])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Embed Chunks
Embed chunks and then load into a Chromadb vector database

In [35]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, \
                                    embedding=embeddings,\
                                    persist_directory="chromadb_store")

## Prompt Engineering

In [41]:
instruction_prompt = (
    "**Analyze:** Carefully examine the provided images and text context.\n"
    "**Synthesize:** Integrate information from both the visual and textual elements.\n"
    "**Reason:** Deduce logical connections and inferences to address the question.\n"
    "**Respond:** Provide a concise, accurate answer in the following format:\n\n"
    "* **Question:** {question}\n"
    "* **Answer:** [Direct response to the question]\n"
    "* **Explanation:** [Bullet-point reasoning steps if applicable]\n"
    "* **Source:** [name of the file, page, image from where the information is cited]\n\n"
    "**Ambiguity:** If the context is insufficient to answer, respond 'Not enough context to answer.'"
)
custom_prompt_template = f"{instruction_prompt}\n\nQuestion: {{question}}\n\nAnswer:"
prompt = PromptTemplate(template=custom_prompt_template, input_variables=["question"])

In [45]:
from langchain.prompts import PromptTemplate

custom_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=(
        "**Analyze:** Carefully examine the provided images and text context.\n"
        "**Synthesize:** Integrate information from both the visual and textual elements.\n"
        "**Reason:** Deduce logical connections and inferences to address the question.\n"
        "**Respond:** Provide a concise, accurate answer in the following format:\n\n"
        "* **Question:** {question}\n"
        "* **Answer:** [Direct response to the question]\n"
        "* **Explanation:** [Bullet-point reasoning steps if applicable]\n"
        "* **Source:** [name of the file, page, image from where the information is cited]\n\n"
        "**Ambiguity:** If the context is insufficient to answer, respond 'Not enough context to answer.'"
    )
)

## RAG Agent
Define RAG agent and pass the vector database

In [47]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.2)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    prompt=custom_prompt_template
)

ValidationError: 1 validation error for RetrievalQA
prompt
  extra fields not permitted (type=value_error.extra)

## Parse querries

In [ ]:


def generate_answer(query, retrieved_docs, openai_llm):
    context = "\n".join(retrieved_docs)
    instruction_prompt = (
        f"**Analyze:** Carefully examine the provided images and text context.\n"
        f"**Synthesize:** Integrate information from both the visual and textual elements.\n"
        f"**Reason:** Deduce logical connections and inferences to address the question.\n"
        f"**Respond:** Provide a concise, accurate answer in the following format:\n\n"
        f"* **Question:** {query}\n"
        f"* **Answer:** [Direct response to the question]\n"
        f"* **Explanation:** [Bullet-point reasoning steps if applicable]\n"
        f"* **Source:** [name of the file, page, image from where the information is cited]\n\n"
        f"**Ambiguity:** If the context is insufficient to answer, respond 'Not enough context to answer.'"
    )
    prompt = f"Context: {context}\n\n{instruction_prompt}\n\nQuestion: {query}\n\nAnswer:"
    response = openai_llm(prompt)
    return response

In [25]:
pdf_dir = '/content/drive/MyDrive/Papers'
pdf_files = [os.path.join(pdf_dir, file) for file in os.listdir(pdf_dir) if file.endswith('.pdf')]
pdf_text = extract_text_from_pdf(pdf_files[0])

In [34]:
def query_pdf(query):
    response = rag_chain.run(query)
    return response

query = "What is the summary of the paper?"
response = query_pdf(query)

The paper discusses the topic of performance capture of humans, which is a highly active area of research. The authors present various methods for performance capture, including image-based methods, model-based approaches, volumetric capture systems, and machine learning-based algorithms. They also introduce a system called "The Relightables", which uses a Light Stage combined with multi-view stereo depth sensors for capture, reconstruction, and rendering. The system is complex and requires high computational power, but the authors argue that it is a significant improvement over the state of the art in terms of speed. The paper also includes an evaluation of the system's main components to justify the design choices.
Meshes per frame are mapped to each other through a process called mesh alignment. For each mesh, a sequential forward alignment and backward alignment through time is performed. This means that the n-th mesh is aligned to all its proceeding meshes when moving forward in t